In [1]:
import cx_list_from_stabilizers_in_sequence
import draw_ordered_tanner_graph
from circuit_from_cx_list import memory_experiment_circuit_from_cx_list
%load_ext autoreload
%autoreload 2

In [2]:
code = cx_list_from_stabilizers_in_sequence.RotatedSurfaceCode(L=5)

cx_list = code.generate_cx_list()
ancilla_type, data_mapping, ancilla_mapping = code.build_mappings()
lz = code.lz
lx = code.lx
data_coords = code.data_coords
ancilla_coords = code.ancilla_coords
qubit_to_name = {v:k for k,v in data_mapping.items()}
qubit_to_name.update({v:k for k,v in ancilla_mapping.items()})

In [ ]:
from permute_within_each_stabilizer import random_permutation_within_each_stabilizer
from permute_single_stabilizer import permute_single_stabilizer_inplace

In [ ]:
random_permutation_within_each_stabilizer(cx_list)


In [ ]:
hook_errors = {ancilla: [(1, 0.01)] for ancilla, type in ancilla_type.items() if type == 'X'}

In [ ]:
for _ in range(20):
    _, circ, idle_time = memory_experiment_circuit_from_cx_list(
                cx_list=cx_list,
                ancilla_type=ancilla_type,
                data_mapping=data_mapping,
                ancilla_mapping=ancilla_mapping,
                flag_mapping=dict(),  # No flag mapping used here
                logicals=code.lz,
                logical_type='Z',
                p_cx=0.,
                p_idle=0.,
                p_measurement_error=0.01,
                p_phenomenological_error=0.01,
                hook_errors=hook_errors,
                cycles_before_noise=1,
                cycles_with_noise=1,
                cycles_after_noise=1,
                flag=False
            )
    errors = circ.search_for_undetectable_logical_errors(
        dont_explore_edges_increasing_symptom_degree=False,
        dont_explore_detection_event_sets_with_size_above=9999,
        dont_explore_edges_with_degree_above=9999,
        canonicalize_circuit_errors=True,
    )
    print(len(errors))
    qubits_in_minimal_error = [error.circuit_error_locations[0].instruction_targets.targets_in_range[0].gate_target.value for error in errors]
    hook_ancillas_in_minimal_error = [qubit_to_name[qubit] for qubit in qubits_in_minimal_error if qubit in ancilla_mapping.values()]
    print(hook_ancillas_in_minimal_error)
    for a in hook_ancillas_in_minimal_error:
        permute_single_stabilizer_inplace(cx_list, a)